In [ ]:
import random
import csv
import time
import os
from deap import base, creator, tools
from codecarbon import EmissionsTracker

# Configuración de experimento
generar_repeticiones = 10  # Ajustar si se desean múltiples corridas por configuración
poblaciones = [2**6, 2**10, 2**14]
cruces = [0.2, 0.01, 0.8]
mutaciones = [0.1]
tamaño_individuo = 1024
max_time = 120  # segundos (ajustable)
output_file = 'resultados_DEAP_SBX_GA.csv'

# Dominio y escala para Sphere
LOW, UP = -5.12, 5.12
F_MAX = tamaño_individuo * (UP ** 2)

# CSV de resultados: siempre sobrescribir al inicio
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        'framework', 'poblacion', 'cruce', 'mutacion',
        'generaciones_executadas', 'fitness_inicial_max(%)',
        'fitness_maximo_alcanzado(%)', 'variacion_fitness(%)',
        'gen_fitness_max', 'motivo_parada', 'tiempo_total_segundos',
        'emisiones_kgCO2eq', 'timestamp'
    ])

# Función para verificar si se ha excedido el tiempo
def tiempo_excedido(tiempo_inicio, tiempo_max):
    return time.time() - tiempo_inicio >= tiempo_max

# Bucle de experimentos
for poblacion in poblaciones:
    for cruce in cruces:
        for mutacion in mutaciones:
            for run_id in range(generar_repeticiones):
                # Limpiar definiciones previas de DEAP para evitar conflictos
                if hasattr(creator, 'FitnessMax'):
                    del creator.FitnessMax
                if hasattr(creator, 'Individual'):
                    del creator.Individual
                
                # Iniciar seguimiento emisiones
                tracker= EmissionsTracker(
                        project_name="Sphere_Experiment",  
                        output_dir=".",
                        log_level="critical",
                        save_to_file=True,
                        tracking_mode="process",
                        allow_multiple_runs=True
                    )
                tracker.start()
                t_start = time.time()

                # Configurar DEAP
                creator.create('FitnessMax', base.Fitness, weights=(1.0,))
                creator.create('Individual', list, fitness=creator.FitnessMax)
                toolbox = base.Toolbox()
                toolbox.register('attr_float', random.uniform, LOW, UP)
                toolbox.register('individual', tools.initRepeat,
                                 creator.Individual, toolbox.attr_float, tamaño_individuo)
                toolbox.register('population', tools.initRepeat, list, toolbox.individual)
                toolbox.register('mate', tools.cxSimulatedBinaryBounded,
                                 low=LOW, up=UP, eta=20.0)
                toolbox.register('mutate', tools.mutPolynomialBounded,
                                 low=LOW, up=UP, eta=20.0, indpb=mutacion)
                toolbox.register('select', tools.selTournament, tournsize=2)

                def evaluate(ind):
                    raw = sum(x * x for x in ind)
                    return ((1 - raw / F_MAX) * 100,)

                toolbox.register('evaluate', evaluate)

                # Población y fitness inicial
                pop = toolbox.population(n=poblacion)
                
                # Verificar tiempo antes de la evaluación inicial
                if tiempo_excedido(t_start, max_time):
                    fitness_init_max = 0
                    fitness_max = 0
                    gen_fitness_max = 0
                    generaciones = 0
                    motivo = 'timeout'
                    tiempo_total = time.time() - t_start
                    emis = tracker.stop()
                    variacion = 0
                    
                    # Guardar resultados y continuar con la siguiente configuración
                    data_row = [
                        'DEAP_SBX', poblacion, cruce, mutacion,
                        generaciones, f'{fitness_init_max:.6f}', f'{fitness_max:.6f}',
                        f'{variacion:.6f}', gen_fitness_max, motivo,
                        f'{tiempo_total:.2f}', f'{emis:.6f}',
                        time.strftime('%Y-%m-%d %H:%M:%S')
                    ]
                    with open(output_file, 'a', newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerow(data_row)
                        csvfile.flush()
                    continue
                
                # Evaluar población inicial (con verificación de tiempo)
                for ind in pop:
                    if tiempo_excedido(t_start, max_time):
                        break
                    ind.fitness.values = toolbox.evaluate(ind)
                
                # Verificar después de la evaluación inicial si se excedió el tiempo
                if tiempo_excedido(t_start, max_time):
                    # Asegurarse de que todos los individuos tengan fitness válido para calcular el máximo
                    for ind in pop:
                        if not ind.fitness.valid:
                            ind.fitness.values = (0,)
                
                # Calcular fitness inicial máximo
                fitness_init_max = max((ind.fitness.values[0] if ind.fitness.valid else 0) for ind in pop)

                # Variables de seguimiento
                generaciones = 0
                fitness_max = fitness_init_max
                gen_fitness_max = 0
                motivo = 'timeout'  # Por defecto, asumimos que se detendrá por tiempo

                # Bandera para controlar la salida del bucle
                tiempo_limite_alcanzado = False

                # Bucle evolutivo
                while not tiempo_limite_alcanzado:
                    # Verificar tiempo al inicio de cada generación
                    if tiempo_excedido(t_start, max_time):
                        tiempo_limite_alcanzado = True
                        break
                    
                    generaciones += 1

                    # Cruce
                    offspring = list(map(toolbox.clone, pop))
                    for i in range(0, len(offspring), 2):
                        # Verificar tiempo antes de cada operación de cruce
                        if tiempo_excedido(t_start, max_time):
                            tiempo_limite_alcanzado = True
                            break
                        
                        # Asegurarse de que hay suficientes individuos para el cruce
                        if i+1 < len(offspring):
                            c1, c2 = offspring[i], offspring[i+1]
                            if random.random() < cruce:
                                toolbox.mate(c1, c2)
                                del c1.fitness.values, c2.fitness.values
                    
                    if tiempo_limite_alcanzado:
                        break
                    
                    # Mutación
                    for m in offspring:
                        # Verificar tiempo antes de cada operación de mutación
                        if tiempo_excedido(t_start, max_time):
                            tiempo_limite_alcanzado = True
                            break
                        
                        if random.random() < mutacion:
                            toolbox.mutate(m)
                            del m.fitness.values
                    
                    if tiempo_limite_alcanzado:
                        break
                    
                    # Evaluar nuevos individuos
                    for ind in offspring:
                        # Verificar tiempo antes de cada evaluación
                        if tiempo_excedido(t_start, max_time):
                            tiempo_limite_alcanzado = True
                            break
                        
                        if not ind.fitness.valid:
                            ind.fitness.values = toolbox.evaluate(ind)
                    
                    if tiempo_limite_alcanzado:
                        break
                    
                    # Verificar tiempo antes de la selección
                    if tiempo_excedido(t_start, max_time):
                        tiempo_limite_alcanzado = True
                        break
                    
                    # Selección
                    pop[:] = toolbox.select(offspring, len(pop))
                    
                    # Verificar tiempo antes de comprobar mejora
                    if tiempo_excedido(t_start, max_time):
                        tiempo_limite_alcanzado = True
                        break
                    
                    # Comprobar mejora y posible convergencia
                    # Asegurarse de que todos los individuos tienen fitness válido
                    valid_pop = [ind for ind in pop if ind.fitness.valid]
                    if valid_pop:
                        curr_best = tools.selBest(valid_pop, 1)[0]
                        curr_fit = curr_best.fitness.values[0]
                        if curr_fit > fitness_max:
                            fitness_max = curr_fit
                            gen_fitness_max = generaciones
                            if fitness_max >= 100.0:
                                motivo = 'convergence'
                                tiempo_limite_alcanzado = True
                                break

                # Calcular tiempo total (asegurando que sea preciso)
                tiempo_total = time.time() - t_start
                
                # Parar tracker y obtener emisiones
                emis = tracker.stop()

                # Cálculo variación
                variacion = fitness_max - fitness_init_max

                # Guardar fila inmediatamente
                data_row = [
                    'DEAP_SBX', poblacion, cruce, mutacion,
                    generaciones, f'{fitness_init_max:.6f}', f'{fitness_max:.6f}',
                    f'{variacion:.6f}', gen_fitness_max, motivo,
                    f'{tiempo_total:.2f}', f'{emis:.6f}',
                    time.strftime('%Y-%m-%d %H:%M:%S')
                ]
                with open(output_file, 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow(data_row)
                    csvfile.flush()

print(f'Resultados guardados en {output_file}')
